# PlaceKitten Demo Notebook 🐱

Interactive demonstration of PlaceKitten's Phase 2 features:
- Computer Vision Pipeline
- Smart Cropping Engine
- Step Visualization System
- Filter Pipeline

## Setup and Imports

In [ ]:
import sys
import os
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import Image, display
import numpy as np

# Add src to path
project_root = Path(os.getcwd()).parent.parent
src_path = project_root / "src"
sys.path.insert(0, str(src_path))

# Create output folder for notebook images
output_folder = Path("notebook_output")
output_folder.mkdir(exist_ok=True)

# Import PlaceKitten
from placekitten import PlaceKitten, list_available_filters

print("✅ PlaceKitten imported successfully!")
print(f"📁 Project root: {project_root}")
print(f"📁 Source path: {src_path}")
print(f"📁 Output folder: {output_folder.absolute()}")

# Helper function to get output path
def get_output_path(filename):
    return str(output_folder / filename)

## 1. Basic PlaceKitten Setup

In [ ]:
# Initialize PlaceKitten
kitten = PlaceKitten("demo")

# Show available images
images = kitten.list_available_images()
count = kitten.get_image_count()

print(f"🐱 Found {count} kitten images:")
for i, img in enumerate(images, 1):  # Show with 1-based indexing
    print(f"  {i}: {img}")

# Show available filters
filters = list_available_filters()
print(f"\n🎨 Available filters: {', '.join(filters)}")

## 2. Basic Image Generation

In [ ]:
# Generate a basic image
processor = kitten.generate(width=600, height=400, image_id=1)  # Now 1-based indexing
basic_output = processor.save(get_output_path("notebook_basic.jpg"))

print(f"✅ Basic image generated: {basic_output}")

# Display the image
display(Image(basic_output))

## 3. Smart Cropping with Computer Vision

This demonstrates the 9-step intelligent cropping process:
1. Original analysis
2. Grayscale conversion
3. Noise reduction (Gaussian blur)
4. Edge detection (Canny algorithm)
5. Contour analysis
6. Subject bounding box
7. Rule of thirds grid
8. Optimal crop area
9. Final result

In [ ]:
# Generate image with smart cropping and step visualization
processor = kitten.generate(width=800, height=600, image_id=2)  # Start with different aspect ratio

# Apply smart cropping to 16:9 format with step visualization
smart_processor = processor.smart_crop(
    width=800, 
    height=450,  # 16:9 aspect ratio
    save_steps=True,
    output_prefix=str(output_folder / "notebook_demo")
)

# Save final result
smart_output = smart_processor.save(get_output_path("notebook_smart_crop.jpg"))
print(f"✅ Smart cropping completed: {smart_output}")

# Display the final result
display(Image(smart_output))

## 4. Step-by-Step Visualization

View each step of the computer vision pipeline:

In [ ]:
# Display all processing steps
step_files = [
    (get_output_path("notebook_demo_1-original.jpg"), "1. Original Image"),
    (get_output_path("notebook_demo_2-grayscale.jpg"), "2. Grayscale Conversion"),
    (get_output_path("notebook_demo_3-blurred.jpg"), "3. Noise Reduction (Gaussian Blur)"),
    (get_output_path("notebook_demo_4-edges.jpg"), "4. Edge Detection (Canny)"),
    (get_output_path("notebook_demo_5-largest-contour.jpg"), "5. Contour Analysis"),
    (get_output_path("notebook_demo_6-bounding-box.jpg"), "6. Subject Bounding Box"),
    (get_output_path("notebook_demo_7-rule-of-thirds.jpg"), "7. Rule of Thirds Grid"),
    (get_output_path("notebook_demo_8-crop-area.jpg"), "8. Optimal Crop Area"),
    (get_output_path("notebook_demo_9-final.jpg"), "9. Final Result")
]

# Create a grid display
fig, axes = plt.subplots(3, 3, figsize=(15, 15))
fig.suptitle('PlaceKitten Smart Crop - Computer Vision Pipeline', fontsize=16)

for i, (filename, title) in enumerate(step_files):
    row = i // 3
    col = i % 3
    
    if os.path.exists(filename):
        img = mpimg.imread(filename)
        axes[row, col].imshow(img)
        axes[row, col].set_title(title, fontsize=10)
        axes[row, col].axis('off')
    else:
        axes[row, col].text(0.5, 0.5, f'File not found:\n{Path(filename).name}', 
                          ha='center', va='center', transform=axes[row, col].transAxes)
        axes[row, col].set_title(title, fontsize=10)
        axes[row, col].axis('off')

plt.tight_layout()
plt.show()

# Show which files were created
created_files = [f for f, _ in step_files if os.path.exists(f)]
print(f"✅ Created {len(created_files)} visualization steps")

## 5. Filter Pipeline Demo

Test all available filters with the same source image:

In [ ]:
# Test all available filters
filters_to_test = ['grayscale', 'sepia', 'blur', 'invert', 'pixelate']
filter_results = {}

base_processor = kitten.generate(width=400, height=300, image_id=3)  # Now 1-based indexing

# Generate original
original_file = base_processor.save(get_output_path("notebook_original.jpg"))
filter_results['original'] = original_file

# Apply each filter
for filter_name in filters_to_test:
    try:
        filtered_processor = base_processor.apply_filter(filter_name)
        output_file = filtered_processor.save(get_output_path(f"notebook_{filter_name}.jpg"))
        filter_results[filter_name] = output_file
        print(f"✅ {filter_name} filter applied")
    except Exception as e:
        print(f"❌ {filter_name} filter failed: {e}")
        filter_results[filter_name] = None

print(f"\n📊 Generated {len([f for f in filter_results.values() if f])} filter variations")

In [ ]:
# Display filter results in a grid
available_filters = [(k, v) for k, v in filter_results.items() if v and os.path.exists(v)]
n_filters = len(available_filters)

if n_filters > 0:
    cols = 3
    rows = (n_filters + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(12, 4 * rows))
    fig.suptitle('PlaceKitten Filter Pipeline Results', fontsize=16)
    
    # Handle single row case
    if rows == 1:
        axes = [axes] if cols == 1 else axes
    
    for i, (filter_name, filepath) in enumerate(available_filters):
        row = i // cols
        col = i % cols
        
        ax = axes[row][col] if rows > 1 else axes[col]
        
        img = mpimg.imread(filepath)
        ax.imshow(img)
        ax.set_title(filter_name.title(), fontsize=12)
        ax.axis('off')
    
    # Hide empty subplots
    for i in range(n_filters, rows * cols):
        row = i // cols
        col = i % cols
        ax = axes[row][col] if rows > 1 else axes[col]
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ No filter results to display")

## 6. Method Chaining Demo

Demonstrate the fluent interface with method chaining:

In [ ]:
# Complex method chaining example
chained_result = (
    kitten.generate(width=800, height=600, image_id=1)  # Now 1-based indexing
    .smart_crop(width=600, height=400, save_steps=False)
    .apply_filter("sepia")
    .save(get_output_path("notebook_chained.jpg"))
)

print(f"✅ Method chaining completed: {chained_result}")
display(Image(chained_result))

# Show the power of chaining
print("\n🔗 This single chain performed:")
print("   1. Generated 800x600 image from kitten #1")
print("   2. Smart cropped to 600x400 with computer vision")
print("   3. Applied sepia filter")
print("   4. Saved the final result")

## 7. Batch Processing Demo

In [ ]:
# Batch processing configuration
batch_configs = [
    {"width": 400, "height": 300, "filter_type": "grayscale", "image_id": 1},  # Now 1-based
    {"width": 500, "height": 281, "filter_type": "sepia", "image_id": 2},
    {"width": 600, "height": 400, "filter_type": "blur", "image_id": 3},
]

# Process batch in output folder
batch_results = kitten.batch_process(batch_configs, str(output_folder / "batch_output"))

print(f"✅ Batch processing completed: {len(batch_results)} images")
for i, result in enumerate(batch_results):
    print(f"   {i+1}. {result}")

# Display batch results
if batch_results:
    fig, axes = plt.subplots(1, len(batch_results), figsize=(15, 5))
    fig.suptitle('Batch Processing Results', fontsize=16)
    
    if len(batch_results) == 1:
        axes = [axes]
    
    for i, result_path in enumerate(batch_results):
        if os.path.exists(result_path):
            img = mpimg.imread(result_path)
            axes[i].imshow(img)
            config = batch_configs[i]
            title = f"{config['width']}x{config['height']}\n{config['filter_type']}"
            axes[i].set_title(title, fontsize=10)
            axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

## 8. Performance and Crop Information

In [ ]:
import time

# Performance test
start_time = time.time()

# Generate and process an image
processor = kitten.generate(width=1200, height=800, image_id=1)  # Now 1-based indexing
smart_processor = processor.smart_crop(width=1920, height=1080, save_steps=False)
result = smart_processor.save(get_output_path("notebook_performance_test.jpg"))

end_time = time.time()
processing_time = end_time - start_time

print(f"⏱️  Processing time: {processing_time:.2f} seconds")
print(f"📊 Generated 1920x1080 image with smart cropping")

# Show crop information if available
if hasattr(smart_processor, 'crop_info'):
    crop_info = smart_processor.crop_info
    print(f"\n🔍 Crop Information:")
    print(f"   Original size: {crop_info.get('original_size', 'N/A')}")
    print(f"   Target size: {crop_info.get('target_size', 'N/A')}")
    print(f"   Crop box: {crop_info.get('crop_box', 'N/A')}")
    print(f"   Subject bbox: {crop_info.get('subject_bbox', 'N/A')}")
    print(f"   Contour area: {crop_info.get('contour_area', 'N/A')}")

# Display final performance test result
if os.path.exists(result):
    display(Image(result))

## 9. Summary

🎉 **PlaceKitten Phase 2 Complete!**

✅ **Computer Vision Pipeline**: Edge detection, contour analysis, subject identification  
✅ **Smart Cropping Engine**: Rule-of-thirds, optimal positioning, boundary safety  
✅ **Step Visualization**: 9-step debug output with processing stages  
✅ **Filter Pipeline**: Multiple filters with method chaining  
✅ **Batch Processing**: Multiple image processing workflows  
✅ **Performance**: Fast processing with detailed crop information  

The PlaceKitten library is ready for Phase 3 (Advanced Features) and Phase 4 (MCP Integration)!

In [ ]:
# Cleanup generated files (optional)
import glob

# List all generated files in output folder
generated_files = list(output_folder.glob("notebook_*")) + list(output_folder.glob("*batch_output*"))
print(f"📁 Generated {len(generated_files)} files during this demo:")
for f in sorted(generated_files):
    print(f"   📄 {f.name}")

print(f"\n📁 All files saved in: {output_folder.absolute()}")

# Uncomment to clean up:
# for f in generated_files:
#     try:
#         f.unlink()
#         print(f"🗑️  Removed {f.name}")
#     except:
#         pass